In [4]:
import pandas as pd
import datapane as dp
import altair as alt
import yfinance as yf

from scipy.stats import zscore

# Pull default parameters from our config file.
dp.Params.load_defaults('./datapane.yaml')

tickers = dp.Params.get('tickers')
plot_type = dp.Params.get('plot_type')
period = dp.Params.get('period')
calc_zscore = dp.Params.get('zscore')
data = yf.download(tickers=' '.join(tickers), period=period, groupby='ticker').Close

# Create charts and df
if calc_zscore:
    for ticker in tickers:
        data[ticker] = zscore(data[ticker])

df = data.reset_index().melt('Date', var_name='symbol', value_name='price')

base_chart = alt.Chart(df).encode(x='Date:T', y='price:Q', color='symbol').interactive()

chart = base_chart.mark_line() if plot_type == 'line' else base_chart.mark_bar()

[*********************100%***********************]  3 of 3 completed


In [10]:
# Create report with chart, df, image, and code

dp.Report(dp.Markdown(f'''
```df = data.reset_index().melt('Date', var_name='symbol', value_name='price')

base_chart = alt.Chart(df).encode(x='Date:T', y='price:Q', color='symbol').interactive()

chart = base_chart.mark_line() if plot_type == 'line' else base_chart.mark_bar()```
'''),
          dp.Plot(chart), 
          dp.Table(df),
          dp.File(file=dp.Path('image.png'))
          ).publish(name=f'stock_report', headline=f'Report on {" ".join(tickers)}')

Publishing report and associated data - please wait..
Report successfully published at https://datapane.com/khuyentran1401/reports/stock_report_bbdd8a75/
